no sys args version, for debug purpose

Following block are complete process for running KMC on API=1, from generating input files to run the kernel, Always run it see if it works after make modification on code.


following block is for generating the local cluster expansion model. See the Table S3 in support information of nasicon kmc as reference

The localclusterexpansion.initialization is for generating the **lce.json and lcesite.json**

In [1]:
from kmcpy.model import LocalClusterExpansion

a=LocalClusterExpansion()
a.initialization(center_Na1_index=0,cutoff_cluster=[6,6,6],cutoff_region=4,template_cif_fname='./EntryWithCollCode15546_Na4Zr2Si3O12_573K.cif')
a.to_json("lce.json")

Searching local env around [ 0.      0.     11.0905] Na ...
793 clusters will be generated ...

Generating possible clusters within this diffusion unit...
Cutoffs: pair = 6 Angst, triplet = 6 Angst, quadruplet = 6 Angst
Type	Index	max_length	min_length	Point Group	Multiplicity
point	(0,)      	   0.000	   0.000	Kh   	    6
point	(6,)      	   0.000	   0.000	Kh   	    6
pair 	(0, 1)    	   4.727	   4.727	D*h  	    6
pair 	(0, 3)    	   5.103	   5.103	D*h  	    6
pair 	(0, 6)    	   3.710	   3.710	C*v  	    6
pair 	(0, 7)    	   3.056	   3.056	C*v  	    6
pair 	(0, 8)    	   3.162	   3.162	C*v  	    6
pair 	(6, 7)    	   4.922	   4.922	D*h  	    6
pair 	(6, 9)    	   5.629	   5.629	D*h  	    6
triplet	(0, 1, 2) 	   5.103	   4.727	C2v  	    6
triplet	(0, 1, 6) 	   4.727	   3.162	Cs   	    6
triplet	(0, 1, 7) 	   4.727	   3.056	Cs   	    6
triplet	(0, 3, 4) 	   5.103	   5.103	D3h  	    2
triplet	(0, 3, 7) 	   5.103	   3.056	C2v  	    6
triplet	(0, 6, 7) 	   4.922	   3.056	Cs   	    6
tripl

the kmc.event_generator is for generating the **event_kernal.csv and events.json**

read the primitive cell from prim.json, and write events.json as output

In [2]:
from kmcpy.event_generator import generate_events

generate_events(prim_fname=r"../examples/inputs/prim.json",supercell_shape=[2,1,1],event_fname="./events.json",event_kernal_fname='./event_kernal.csv')

Generating events ...
Initializing model with pirm.json at ../examples/inputs/prim.json ...
Supercell Shape:
 [[2 0 0]
 [0 1 0]
 [0 0 1]]
Converting supercell ...
Looking for all possible swaps with ncpus = 40  (this might take a while) ...
Saving: ./events.json
Generating event kernal ...
Saving into: ./event_kernal.csv


Following block run the kmc version 1

In [3]:


from kmcpy.io import InputSet,load_occ
from kmcpy.kmc import KMC
import os
import sys

def main(api=1,**kwargs):
    os.chdir("../examples")
    inputset=InputSet.from_json("./test_input.json")

    print(inputset._parameters.keys())
    inputset.parameter_checker()

    if api==1:


        inputset.set_parameter("occ",load_occ(inputset._parameters["mc_results"],inputset._parameters["supercell_shape"],api=inputset.api))
        # step 1 initialize global occupation and conditions
        kmc = KMC()
        events_initialized = kmc.initialization(**inputset._parameters) # v in 10^13 hz

        # # step 2 compute the site kernal (used for kmc run)
        kmc.load_site_event_list(inputset._parameters["event_kernel"])

        # # step 3 run kmc
        kmc.run_from_database(events=events_initialized,**inputset._parameters)
    if api>1:
        raise NotImplementedError
    pass


if __name__ == "__main__":

    main()


dict_keys(['v', 'equ_pass', 'kmc_pass', 'supercell_shape', 'fitting_results', 'fitting_results_site', 'lce_fname', 'lce_site_fname', 'prim_fname', 'event_fname', 'event_kernel', 'mc_results', 'T', 'comp', 'structure_idx', 'api'])
Initializing kMC calculations with pirm.json at ./inputs/prim.json ...
Supercell Shape:
 [[2 0 0]
 [0 1 0]
 [0 0 1]]
Converting supercell ...
Loading fitting results ...
Loading fitting results (site energy) ...
Loading occupation: [-1 -1 -1 -1 -1 -1 -1 -1  1 -1 -1 -1 -1 -1 -1  1 -1  1 -1 -1  1 -1 -1 -1
 -1 -1 -1 -1]
Fitted time and error (LOOCV,RMS)
01/21/2022, 16:49:17 149.8671381416 29.1495050791
Fitted KECI and empty cluster
[10.5029085379, 4.0803809727, 0.0, 0.0, -11.2639080302, 0.0, -4.0578126572, 0.0, -7.2124075988, 0.0, 0.0, 0.0, 0.0] 339.1083600548
Fitted time and error (LOOCV,RMS) (site energy)
01/21/2022, 16:49:17 149.8671381416 29.1495050791
Fitted KECI and empty cluster (site energy)
[10.5029085379, 4.0803809727, 0.0, 0.0, -11.2639080302, 0.0, -4.

/userhome1/david/repo/kmcPy/kmcpy/tracker.py:93: RuntimeWarning: invalid value encountered in true_divide
  corr_factor = np.linalg.norm(self.displacement,axis=1)**2/(self.hop_counter*a**2)


0	2.067E-08	5.315E+00	5.967E-09	4.286E-09	4.822E-01	7.184E-01	2.161E-01	0.29	0.24
1	8.320E-08	1.396E+01	8.853E-09	2.796E-09	7.155E-01	3.158E-01	4.217E-01	0.29	0.24
2	1.360E-07	1.396E+01	5.415E-09	1.710E-09	4.376E-01	3.158E-01	3.539E-01	0.29	0.24
3	2.487E-07	1.569E+01	4.242E-09	1.051E-09	3.428E-01	2.478E-01	4.096E-01	0.29	0.24
4	3.281E-07	2.140E+01	5.310E-09	1.087E-09	4.291E-01	2.048E-01	4.553E-01	0.29	0.24
5	4.159E-07	1.829E+01	3.246E-09	7.327E-10	2.623E-01	2.257E-01	4.041E-01	0.29	0.24
6	8.453E-07	2.312E+01	1.894E-09	4.559E-10	1.530E-01	2.408E-01	3.955E-01	0.29	0.24
7	9.192E-07	2.788E+01	1.304E-09	5.055E-10	1.054E-01	3.876E-01	3.404E-01	0.29	0.24
8	9.667E-07	3.125E+01	1.451E-09	5.388E-10	1.173E-01	3.713E-01	3.421E-01	0.29	0.24
9	9.954E-07	2.628E+01	1.135E-09	4.401E-10	9.176E-02	3.876E-01	3.168E-01	0.29	0.24
10	1.033E-06	2.770E+01	1.424E-09	4.469E-10	1.151E-01	3.138E-01	3.132E-01	0.29	0.24
11	1.096E-06	3.125E+01	1.280E-09	4.751E-10	1.034E-01	3.713E-01	3.172E-01	0.29	0.24
12	1.176E-06	3

Api version 2 initiated
aiming for more flexible inputs.
Following block is the main KMC function
DO make sure that they provide similar result as in API=1
KMC result is not necessarily the same, since it is randomized

In [4]:
#!/usr/bin/env python

from kmcpy.io import InputSet,load_occ
from kmcpy.kmc import KMC
import os
import sys

def main(api=2,**kwargs):
    inputset=InputSet.from_json("test_input_v2.json",api=2)

    print(inputset._parameters.keys())
    inputset.parameter_checker()

    inputset.set_parameter("occ",load_occ(fanme=inputset._parameters["mc_results"],shape=inputset._parameters["supercell_shape"],select_sites=inputset._parameters["select_sites"],api=inputset.api,verbose=True))
    
    
    

    # step 1 initialize global occupation and conditions
    
    
    
    kmc = KMC(api=api)
    events_initialized = kmc.initialization(**inputset._parameters) # v in 10^13 hz

    # # step 2 compute the site kernal (used for kmc run)
    kmc.load_site_event_list(inputset._parameters["event_kernel"])

    # # step 3 run kmc
    kmc.run_from_database(events=events_initialized,**inputset._parameters)


if __name__ == "__main__":

    main()

dict_keys(['v', 'equ_pass', 'kmc_pass', 'supercell_shape', 'fitting_results', 'fitting_results_site', 'lce_fname', 'lce_site_fname', 'prim_fname', 'event_fname', 'event_kernel', 'mc_results', 'select_sites', 'T', 'comp', 'structure_idx', 'api', 'immutable_sites', 'verbose', 'random_seed', 'use_numpy_random_kernel'])
verbose output is enabled, this is function io.load_acc from Api2
selected sites are [0, 1, 2, 3, 4, 5, 6, 7, 12, 13, 14, 15, 16, 17]
converting the occupation raw data to dimension: (42, 2, 1, 1)
occupation_chebyshev: [-1 -1 -1 -1 -1 -1 -1 -1  1 -1 -1 -1 -1 -1 -1  1 -1  1 -1 -1  1 -1 -1 -1
 -1 -1 -1 -1]
Initializing kMC calculations with pirm.json at ./inputs/prim.json ...
Supercell Shape:
 [[2 0 0]
 [0 1 0]
 [0 0 1]]
Converting to the supercell ...
removing the immutable sites: ['Zr', 'O']
Loading fitting results E_kra...
Loading fitting results (site energy or E_end) ...
Loading occupation: [-1 -1 -1 -1 -1 -1 -1 -1  1 -1 -1 -1 -1 -1 -1  1 -1  1 -1 -1  1 -1 -1 -1
 -1 -1 -

/userhome1/david/repo/kmcPy/kmcpy/tracker.py:93: RuntimeWarning: invalid value encountered in true_divide
  corr_factor = np.linalg.norm(self.displacement,axis=1)**2/(self.hop_counter*a**2)


17	2.089E-06	3.806E+01	1.242E-09	3.037E-10	1.004E-01	2.445E-01	1.360E-01	0.29	0.24
18	2.385E-06	4.593E+01	1.884E-09	3.210E-10	1.523E-01	1.703E-01	1.644E-01	0.29	0.24
19	2.706E-06	5.183E+01	1.784E-09	3.192E-10	1.442E-01	1.789E-01	1.622E-01	0.29	0.24
20	2.780E-06	5.670E+01	2.143E-09	3.399E-10	1.732E-01	1.587E-01	1.597E-01	0.29	0.24
21	2.829E-06	7.727E+01	2.686E-09	4.552E-10	2.171E-01	1.695E-01	2.044E-01	0.29	0.24
22	2.898E-06	7.225E+01	2.440E-09	4.155E-10	1.972E-01	1.703E-01	1.779E-01	0.29	0.24
23	2.992E-06	8.916E+01	2.959E-09	4.967E-10	2.391E-01	1.679E-01	2.078E-01	0.29	0.24
24	3.137E-06	9.589E+01	3.316E-09	5.094E-10	2.680E-01	1.536E-01	2.216E-01	0.29	0.24
25	3.315E-06	1.041E+02	3.163E-09	5.232E-10	2.556E-01	1.654E-01	2.163E-01	0.29	0.24
26	3.358E-06	9.498E+01	2.823E-09	4.714E-10	2.281E-01	1.670E-01	1.879E-01	0.29	0.24
27	3.475E-06	1.041E+02	3.084E-09	4.990E-10	2.492E-01	1.618E-01	1.950E-01	0.29	0.24
28	3.528E-06	9.745E+01	2.752E-09	4.603E-10	2.224E-01	1.672E-01	1.832E-01	0.29	0.24
29	3

testing the sites_and_lattices.py

In [18]:
from kmcpy.sites_and_lattices import PrimitiveCell,Site,get_rotation_matrices_from_pymatgen

get_rotation_matrices_from_pymatgen()

primitivecell=PrimitiveCell.from_sites_and_symmetry_matrices(symmetry_operations=get_rotation_matrices_from_pymatgen(),initial_sites=[Site(tag="Na1",relative_coordinate_in_cell=[0.0,0.0,0.0]),Site(tag="Na2",relative_coordinate_in_cell=[0.63967, 0., 0.25]),Site(tag="Si",relative_coordinate_in_cell=[0.29544,0.,0.25])])

for site in primitivecell.sites:
    print(site.tag,site.relative_coordinate_in_cell,site.wyckoff_sequence)

Na1 [0.0, 0.0, 0.0] 0
Na1 [0.3333333333333333, 0.6666666666666666, 0.16666666666666666] 6
Na1 [0.6666666666666666, 0.3333333333333333, 0.3333333333333333] 12
Na1 [0.0, 0.0, 0.5] 18
Na1 [0.3333333333333333, 0.6666666666666666, 0.6666666666666666] 24
Na1 [0.6666666666666666, 0.3333333333333333, 0.8333333333333334] 30
Na2 [0.63967, 0.0, 0.25] 0
Na2 [0.36033000000000004, 0.0, 0.75] 1
Na2 [0.0, 0.63967, 0.25] 2
Na2 [0.0, 0.36033000000000004, 0.75] 3
Na2 [0.36033000000000004, 0.36033000000000004, 0.25] 4
Na2 [0.63967, 0.63967, 0.75] 5
Na2 [0.3333333333333333, 0.3063366666666667, 0.9166666666666666] 6
Na2 [0.3333333333333333, 0.02699666666666667, 0.41666666666666663] 7
Na2 [0.9730033333333332, 0.6666666666666666, 0.9166666666666666] 8
Na2 [0.6936633333333333, 0.6666666666666666, 0.41666666666666663] 9
Na2 [0.6936633333333333, 0.02699666666666667, 0.9166666666666666] 10
Na2 [0.9730033333333332, 0.3063366666666667, 0.41666666666666663] 11
Na2 [0.3063366666666667, 0.3333333333333333, 0.583333333

In [1]:
from kmcpy.sites_and_lattices import PrimitiveCell,Site,get_rotation_matrices_from_pymatgen

primitivecell=PrimitiveCell.from_sites_and_symmetry_matrices(symmetry_operations=get_rotation_matrices_from_pymatgen(),initial_sites=[Site(tag="Na1",relative_coordinate_in_cell=[0.0,0.0,0.0]),Site(tag="Na2",relative_coordinate_in_cell=[0.63967, 0., 0.25]),Site(tag="Si",relative_coordinate_in_cell=[0.29544,0.,0.25])])

for j in primitivecell.sites:
    print(j.tag,j.relative_coordinate_in_cell,j.wyckoff_sequence)


from pymatgen.core import Structure
from pymatgen.analysis.local_env import CutOffDictNN
import numpy as np
structure=Structure.from_file("EntryWithCollCode15546_Na4Zr2Si3O12_573K.cif")
print(structure)

local_env_cutoff_dict = {('Na+','Na+'):4,('Na+','Si4+'):4}
local_env_finder = CutOffDictNN(local_env_cutoff_dict)
local_env_info_list =local_env_finder.get_nn_info(structure,1)
for i in local_env_info_list:
    print(i["site"],i["site"].frac_coords,np.array(i["site"].image),i["site"].frac_coords-np.floor(i["site"].frac_coords))
    for j in primitivecell.sites:
        
        if np.allclose(i["site"].frac_coords-np.floor(i["site"].frac_coords),np.array(j.relative_coordinate_in_cell)):
            print("found corresponding",j.tag,j.wyckoff_sequence)
            break
local_env_info_list =local_env_finder.get_nn_info(structure,2)
for i in local_env_info_list:
    print(i["site"],i["site"].frac_coords,np.array(i["site"].image),i["site"].frac_coords-np.floor(i["site"].frac_coords))
    print(primitivecell.what_is_it_at(i["site"].frac_coords-np.floor(i["site"].frac_coords),raise_error=True))
 
        #print("not found")
        
        


Na1 [0.0, 0.0, 0.0] 0
Na1 [0.3333333333333333, 0.6666666666666666, 0.16666666666666666] 1
Na1 [0.6666666666666666, 0.3333333333333333, 0.3333333333333333] 2
Na1 [0.0, 0.0, 0.5] 3
Na1 [0.3333333333333333, 0.6666666666666666, 0.6666666666666666] 4
Na1 [0.6666666666666666, 0.3333333333333333, 0.8333333333333334] 5
Na2 [0.63967, 0.0, 0.25] 0
Na2 [0.36033000000000004, 0.0, 0.75] 1
Na2 [0.0, 0.63967, 0.25] 2
Na2 [0.0, 0.36033000000000004, 0.75] 3
Na2 [0.36033000000000004, 0.36033000000000004, 0.25] 4
Na2 [0.63967, 0.63967, 0.75] 5
Na2 [0.3333333333333333, 0.3063366666666667, 0.9166666666666666] 6
Na2 [0.3333333333333333, 0.02699666666666667, 0.41666666666666663] 7
Na2 [0.9730033333333332, 0.6666666666666666, 0.9166666666666666] 8
Na2 [0.6936633333333333, 0.6666666666666666, 0.41666666666666663] 9
Na2 [0.6936633333333333, 0.02699666666666667, 0.9166666666666666] 10
Na2 [0.9730033333333332, 0.3063366666666667, 0.41666666666666663] 11
Na2 [0.3063366666666667, 0.3333333333333333, 0.5833333333333

In [1]:
from kmcpy.sites_and_lattices import nearest_neighbor_analyzer

analyzer=nearest_neighbor_analyzer()

print(analyzer.structure[0].frac_coords)

for site in analyzer.reference_structure.sites:
    print(site.relative_coordinate_in_cell,site.tag,site.wyckoff_sequence)

print(analyzer.find(index_of_center_atom_of_pymatgen_structure=0))

[0.  0.  0.5]
[0.0, 0.0, 0.0] Na1 0
[0.3333333333333333, 0.6666666666666666, 0.16666666666666666] Na1 1
[0.6666666666666666, 0.3333333333333333, 0.3333333333333333] Na1 2
[0.0, 0.0, 0.5] Na1 3
[0.3333333333333333, 0.6666666666666666, 0.6666666666666666] Na1 4
[0.6666666666666666, 0.3333333333333333, 0.8333333333333334] Na1 5
[0.63967, 0.0, 0.25] Na2 0
[0.36033000000000004, 0.0, 0.75] Na2 1
[0.0, 0.63967, 0.25] Na2 2
[0.0, 0.36033000000000004, 0.75] Na2 3
[0.36033000000000004, 0.36033000000000004, 0.25] Na2 4
[0.63967, 0.63967, 0.75] Na2 5
[0.3333333333333333, 0.3063366666666667, 0.9166666666666666] Na2 6
[0.3333333333333333, 0.02699666666666667, 0.41666666666666663] Na2 7
[0.9730033333333332, 0.6666666666666666, 0.9166666666666666] Na2 8
[0.6936633333333333, 0.6666666666666666, 0.41666666666666663] Na2 9
[0.6936633333333333, 0.02699666666666667, 0.9166666666666666] Na2 10
[0.9730033333333332, 0.3063366666666667, 0.41666666666666663] Na2 11
[0.3063366666666667, 0.3333333333333333, 0.583

In [2]:
# understanding event_generator

from kmcpy.event import Event
import numpy as np

from kmcpy.io import convert
def generate_events2(prim_fname="prim.json",supercell_shape=[2,1,1],event_fname="events.json",event_kernal_fname='event_kernal.csv'):
    import json
    from pymatgen.core.structure import Structure
    from pymatgen.analysis.local_env import CutOffDictNN
    from pymatgen.core.lattice import Lattice
    import multiprocessing
    from joblib import Parallel, delayed
    
    print('Generating events ...')
    shape = supercell_shape
    
    print('Initializing model with pirm.json at',prim_fname,'...')
    with open(prim_fname,'r') as f:
        prim = json.load(f)
        prim_coords = [site['coordinate'] for site in prim['basis']]
        prim_specie_cases = [site['occupant_dof'] for site in prim['basis']]
        prim_lattice = Lattice(prim['lattice_vectors'])
        prim_species = [s[0] for s in prim_specie_cases]# need to change: now from [Na,Li] it will choose Na
        
    supercell_shape_matrix = np.diag(supercell_shape)
    print('Supercell Shape:\n',supercell_shape_matrix)
    structure = Structure(prim_lattice,prim_species,prim_coords)
    print('Converting supercell ...')
    structure.remove_species(['Zr','O'])
    structure.make_supercell(supercell_shape_matrix)
    structure.to(fmt='cif',filename='supercell.cif')
    # np.savetxt('frac_coords.txt',np.array(structure.frac_coords))
    # np.savetxt('cart_coords.txt',np.array(structure.cart_coords))
    # print(structure.lattice.matrix)
    # np.savetxt('latt.txt',np.array(structure.lattice.matrix))
    
    # change the parameter name to center_site (center_atom). 
    # this information  can be loaded from input (future feature)
    # currently, the sodium1 is the first two sites in prim.json
    # need to generalize to arbitrary sequence
    # future code: the input shall be a list, telling the index of center atom / Na1
    # current value 2, equivalent [0,1] (len([0,1])=2)
    #  future input [0,1] or any indices
    
    n_na1_per_prim = 2# number of Na1 per primitive cell
    
    
    n_na1 = n_na1_per_prim*np.prod(shape) # total number of Na1 in the supercell. This should be equal to the empty lines in the event_kernal.csv
    
    
    local_env_cutoff_dict = {('Na','Na'):4,('Na','Si'):4}
    ncpus = multiprocessing.cpu_count()
    print('Looking for all possible swaps with ncpus =',ncpus,' (this might take a while) ...')
    na_1_sites = structure[0:n_na1]
    local_env_finder = CutOffDictNN(local_env_cutoff_dict)
    
    # get_nn_info; return the neiarest neighbor (or local environment)
    local_env_info_list = list(Parallel(n_jobs=ncpus)(delayed(local_env_finder.get_nn_info)(structure,i) for i in np.arange(0,n_na1)))
    # change to scipy.spatial.cKDTtree in the future
    # get_nn_info return list of these thing:
    # local_env_info_list[0]: local environmental information of 0th Na1
    # local_env_info_list[0] includes (or get_nn_info return) list of neighbors, with each neighbor as dictionary of: {site, image(periodic structure index),weight,site_index}     
    
    events = []
    events_dict = []
    for (na1_index,(na1_site,local_env_info)) in enumerate(zip(na_1_sites,local_env_info_list)):
        for local_env in local_env_info:
            # only consider the environment of Na, but not S or P.
            # need generalization for other system.
            # "Na" -> specie_to_diffuse
            # need to be input.
            # probably become a list.
            if "Na" in local_env['site'].specie.symbol:
                this_event = Event()
                this_event.initialization(na1_index,local_env['site_index'],local_env_info)
                events.append(this_event)
                events_dict.append(this_event.as_dict())

ModuleNotFoundError: No module named 'numba'

In [2]:
from kmcpy.sites_and_lattices import nearest_neighbor_analyzer,PrimitiveCell
from pymatgen.core import Structure


analyzer=nearest_neighbor_analyzer(original_structure=Structure.from_file("EntryWithCollCode15546_Na4Zr2Si3O12_573K.cif"), local_env_cutoff_dict = {('Na+','Na+'):4,('Na+','Si4+'):4},reference_structure=PrimitiveCell.from_sites_and_symmetry_matrices(symmetry_operations=get_rotation_matrices_from_pymatgen("EntryWithCollCode15546_Na4Zr2Si3O12_573K.cif"),initial_sites=[Site(tag="Na1",relative_coordinate_in_cell=[0.0,0.0,0.0]),Site(tag="Na2",relative_coordinate_in_cell=[0.63967, 0., 0.25]),Site(tag="Si",relative_coordinate_in_cell=[0.29544,0.,0.25])]))

analyzer.create_supercell(indices_of_center_atom=[0,1,2,3,4,5],center_atom_tag="Na1",diffuse_to="Na2",environment=["Na2","Si"],supercell_shape=[2,3,1],event_fname="events.json",event_kernal_fname='event_kernal.csv')

ModuleNotFoundError: No module named 'numba'